In [1]:
from ib_insync import *
import logging
import random
import itertools

In [2]:
util.startLoop()

In [3]:
ib = IB()
ib.connect('127.0.0.1', 4002, readonly=True, clientId=random.randint(1, 100000))

<IB connected to 127.0.0.1:4002 clientId=61144>

In [4]:
underlying = Stock('MSFT', exchange='SMART', currency='USD')
ib.qualifyContracts(underlying)

[Stock(conId=272093, symbol='MSFT', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='MSFT', tradingClass='NMS')]

In [5]:
option_chains = ib.reqSecDefOptParams(
    underlyingSymbol=underlying.symbol,
    futFopExchange=underlying.exchange if underlying.secType == "FUT" else "",
    underlyingSecType=underlying.secType,
    underlyingConId=underlying.conId,
)

util.df(option_chains)

,exchange,underlyingConId,tradingClass,multiplier,expirations,strikes
0,AMEX,272093,MSFT,100,"[20191220, 20191227, 20200103, 20200110, 20200...","[37.5, 40.0, 42.5, 45.0, 47.5, 50.0, 55.0, 60...."
1,BOX,272093,MSFT,100,"[20191220, 20191227, 20200103, 20200110, 20200...","[37.5, 40.0, 42.5, 45.0, 47.5, 50.0, 55.0, 60...."
2,CBOE2,272093,MSFT,100,"[20191220, 20191227, 20200103, 20200110, 20200...","[37.5, 40.0, 42.5, 45.0, 47.5, 50.0, 55.0, 60...."
3,BATS,272093,MSFT,100,"[20191220, 20191227, 20200103, 20200110, 20200...","[37.5, 40.0, 42.5, 45.0, 47.5, 50.0, 55.0, 60...."
4,EMERALD,272093,MSFT,100,"[20191220, 20191227, 20200103, 20200110, 20200...","[37.5, 40.0, 42.5, 45.0, 47.5, 50.0, 55.0, 60...."
5,PHLX,272093,MSFT,100,"[20191220, 20191227, 20200103, 20200110, 20200...","[37.5, 40.0, 42.5, 45.0, 47.5, 50.0, 55.0, 60...."
6,NASDAQOM,272093,MSFT,100,"[20191220, 20191227, 20200103, 20200110, 20200...","[37.5, 40.0, 42.5, 45.0, 47.5, 50.0, 55.0, 60...."
7,PSE,272093,MSFT,100,"[20191220, 20191227, 20200103, 20200110, 20200...","[37.5, 40.0, 42.5, 45.0, 47.5, 50.0, 55.0, 60...."
8,NASDAQBX,272093,MSFT,100,"[20191220, 20191227, 20200103, 20200110, 20200...","[37.5, 40.0, 42.5, 45.0, 47.5, 50.0, 55.0, 60...."
9,SMART,272093,MSFT,100,"[20191220, 20191227, 20200103, 20200110, 20200...","[37.5, 40.0, 42.5, 45.0, 47.5, 50.0, 55.0, 60...."


In [6]:
limited_option_chains = [c for c in option_chains]
util.df(limited_option_chains)

,exchange,underlyingConId,tradingClass,multiplier,expirations,strikes
0,AMEX,272093,MSFT,100,"[20191220, 20191227, 20200103, 20200110, 20200...","[37.5, 40.0, 42.5, 45.0, 47.5, 50.0, 55.0, 60...."
1,BOX,272093,MSFT,100,"[20191220, 20191227, 20200103, 20200110, 20200...","[37.5, 40.0, 42.5, 45.0, 47.5, 50.0, 55.0, 60...."
2,CBOE2,272093,MSFT,100,"[20191220, 20191227, 20200103, 20200110, 20200...","[37.5, 40.0, 42.5, 45.0, 47.5, 50.0, 55.0, 60...."
3,BATS,272093,MSFT,100,"[20191220, 20191227, 20200103, 20200110, 20200...","[37.5, 40.0, 42.5, 45.0, 47.5, 50.0, 55.0, 60...."
4,EMERALD,272093,MSFT,100,"[20191220, 20191227, 20200103, 20200110, 20200...","[37.5, 40.0, 42.5, 45.0, 47.5, 50.0, 55.0, 60...."
5,PHLX,272093,MSFT,100,"[20191220, 20191227, 20200103, 20200110, 20200...","[37.5, 40.0, 42.5, 45.0, 47.5, 50.0, 55.0, 60...."
6,NASDAQOM,272093,MSFT,100,"[20191220, 20191227, 20200103, 20200110, 20200...","[37.5, 40.0, 42.5, 45.0, 47.5, 50.0, 55.0, 60...."
7,PSE,272093,MSFT,100,"[20191220, 20191227, 20200103, 20200110, 20200...","[37.5, 40.0, 42.5, 45.0, 47.5, 50.0, 55.0, 60...."
8,NASDAQBX,272093,MSFT,100,"[20191220, 20191227, 20200103, 20200110, 20200...","[37.5, 40.0, 42.5, 45.0, 47.5, 50.0, 55.0, 60...."
9,SMART,272093,MSFT,100,"[20191220, 20191227, 20200103, 20200110, 20200...","[37.5, 40.0, 42.5, 45.0, 47.5, 50.0, 55.0, 60...."


In [7]:
option_contracts = [
    (FuturesOption if underlying.secType == "FUT" else Option)(
        symbol=underlying.symbol,
        lastTradeDateOrContractMonth=expiration,
        strike=strike,
        right=right,
        exchange=chain.exchange,
        multiplier=chain.multiplier,
        tradingClass=chain.tradingClass,
    )
    for chain in limited_option_chains
    if chain.exchange == underlying.exchange
    for expiration in chain.expirations
    for strike in chain.strikes
    for right in ["P", "C"]
]

len(option_contracts)

2556

In [8]:
logging.getLogger('ib_insync').level = logging.CRITICAL

In [9]:
qualified_contracts = ib.qualifyContracts(*option_contracts)
len(qualified_contracts)

1182

In [10]:
logging.getLogger('ib_insync').level = logging.INFO

In [11]:
def onBarUpdate(bars, hasNewBar):
    print(bars.contract, bars[-1])

In [16]:
all_bars = []
for contract in qualified_contracts:
    bars = ib.reqRealTimeBars(contract, 5, 'TRADES', False)
    bars.updateEvent += onBarUpdate
    
    ib.sleep(0)
    all_bars.append(bars)

In [17]:
len(all_bars)

1182

In [18]:
ib.sleep(60)

True

In [19]:
sum((len(bars) for bars in all_bars))

0

In [20]:
for bars in all_bars:
    ib.cancelRealTimeBars(bars)